In [5]:
import os
import pandas as pd
import numpy as np

In [6]:
STATIONS = "Station_Data_Extract_Pipe_Delimited_CleanData_2022.txt"

DATASETS= "converted/"

In [7]:
override_dtype = {
    'State_Id':str,
    'Station_Id':str,
    'Travel_Dir':str,
    'Travel_Lane':str,
    'State_Code':str,
    'Month_Record':str,
    'Day_Record':str,
}

In [8]:
stations = pd.read_csv(STATIONS, delimiter="|", dtype=override_dtype) 
# stations

stations.select_dtypes(include='object').dtypes


C:\Users\hec_m\AppData\Local\Temp\ipykernel_15220\3405064016.py:1: DtypeWarning: Columns (30) have mixed types. Specify dtype option on import or set low_memory=False.
  stations = pd.read_csv(STATIONS, delimiter="|", dtype=override_dtype)


Record_Type            object
State_Code             object
Station_Id             object
Travel_Dir             object
Travel_Lane            object
F_System               object
Sample_Type_Volume     object
Sample_Type_Class      object
Algorithm_Volume       object
Sample_Type_Truck      object
Calibration            object
Type_Sensor_1          object
Type_Sensor_2          object
Primary_Purpose        object
LRS_Id                 object
LRS_Point              object
SHRP_Id                object
Prev_Station_Id        object
Year_Established       object
Year_Discontinued      object
Is_Sample              object
Sample_Id              object
NHS                    object
Posted_Signed_Route    object
Con_Signed_Route       object
Station_Location       object
dtype: object

In [9]:
stations

,Record_Type,State_Code,Station_Id,Travel_Dir,Travel_Lane,Year_Record,F_System,Num_Lanes,Sample_Type_Volume,Num_Lanes_Volume,...,Year_Discontinued,County_Code,Is_Sample,Sample_Id,NHS,Posted_Route_Signing,Posted_Signed_Route,Con_Route_Signing,Con_Signed_Route,Station_Location
0,S,01,000002,1,1,22,3R,2,T,2,...,,71,N,,Y,2,00000002,0,,AL2 btw CR5 and Willow St ...
1,S,01,000010,5,1,22,3R,2,T,2,...,,97,N,,Y,2,00000013,0,,"In the Town of Mount Vernon, AL 13 (U ..."
2,S,01,000013,7,1,22,3R,2,T,2,...,,33,N,,Y,2,00000002,0,,"Near the City of Tuscumbia, AL 2 (US ..."
3,S,01,000016,5,1,22,3R,2,T,2,...,,79,N,,Y,3,00000157,0,,"In the City of Moulton, AL 157 betwee ..."
4,S,01,000016,5,2,22,3R,2,T,2,...,,79,N,,Y,3,00000157,0,,"In the City of Moulton, AL 157 betwee ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35630,S,56,0120NE,7,1,22,3U,2,T,2,...,,37,Y,U1900 000000,N,3,530,0,,WY 530 AT SOUTH END OF VIADUCT ...
35631,S,56,0124NE,7,1,22,4U,1,T,1,...,,37,N,000000000000,N,7,00000000,0,00000000,WALNUT ST NE OF S SIDE BELT LOOP ...
35632,S,56,0128WV,3,1,22,3R,2,T,2,...,,13,Y,00P 30 92000,Y,2,00000026,2,00000287,US 26/287 WEST OF DIVERSION DA ...
35633,S,56,0169NW,5,1,22,4R,1,T,1,...,00,45,N,000000000000,N,3,00000585,0,00000000,WY 585 NORTHWEST OF FOUR CORNE ...


In [10]:
def print_ID_ratio():
    ids = len(set (stations["Station_Id"]))
    print (f"{ids}/{len(stations)} = {ids/len(stations)*100.0}%")

    ids_state = set([f"{row['Station_Id']}@{row['Travel_Lane']}({row['Travel_Dir']})@{row['State_Code']}" for i, row in stations.iterrows() ])
    ids_state_c = len(ids_state)
    print (f"{ids_state_c}/{len(stations)} = {ids_state_c/len(stations)*100.0}%")
    return ids_state



def getCoords():
    coords = {}
    ids = len(set (stations["Station_Id"]))
    print (f"{ids}/{len(stations)} = {ids/len(stations)*100.0}%")
    for i, row in stations.iterrows():
        if not isinstance(row['State_Code'], str) or len(row['State_Code'])!=2:
            print(row['State_Code'])
            break
        
        id = row['Station_Id']+"@"+row['State_Code']
        coords[id] = (row['Longitude'], row['Latitude'])
    return coords
# ids = print_ID_ratio()
coords = getCoords()







7290/35635 = 20.457415462326363%


In [11]:
from collections import defaultdict

def processRow(row):
    hours = [f"Hour_{hour:02}" for hour in range(24)]
    total = np.sum(row[hours])

    id = row['Station_Id']+"@"+row['State_Code']
    return id,total

import json

def getDataset(STATE):

    data = {}
    cnt=0
    for itr, filename in enumerate(os.listdir(DATASETS)):
        if not filename.endswith(".VOL"): 
            continue
        if not filename.startswith(STATE): 
            continue
        file_path = os.path.join(DATASETS, filename)
        cnt+=1
        print(f"{cnt}/{12}", filename, end="\r")
        # Read CSV using pandas
        df = pd.read_csv(file_path, dtype=override_dtype)
        # Do whatever operations you want with the dataframe
        for idx, row in df.iterrows():
            id, count = processRow(row)
            if id not in data:
                # If key doesn't exist, create a new empty list for that key
                data[id] = {}
            date = row["Month_Record"]+"/"+row["Day_Record"]
            if date not in data[id]:
                data[id][date]=0
            data[id][date]+=count
            # if (len(data[id])>12)
            #     print(f"{id} has more entries than expected, {len(data[id])}")
    final_data= []
    for idx, arr in data.items():
        if idx not in coords:
            continue
        final_data.append([idx, np.average([val for val in arr.values()]), len(arr), np.sum([val for val in arr.values()]), coords[idx][0],coords[idx][1] ])
    
    

    return pd.DataFrame(final_data, columns=["id","average", "count", "total", "longitude", "latitude"]).set_index("id")
            

STATES = ["AK", "AL", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA" , "HI", "IA","ID", "IL", "IN", "KS", "KY", "LA", "MA","MD","ME", "MI", "MN","MO","MS","NC","NE","NH","NJ","NM", "NV","NY","OH","OK","OR","PA","RI", "SC", "SD", "TN","TX","UT","VA","VT","WA","WI","WV"]

fullDF = None

for state in STATES:
    stationCount = getDataset(state)
    stationCount.to_csv(f"counts/{state}.csv")
    if fullDF is None:
        fullDF = stationCount
    else:
        fullDF = pd.concat([fullDF, stationCount])
        
    

fullDF.to_csv("counts/full_count.csv")


fullDF


C:\Users\hec_m\AppData\Local\Temp\ipykernel_15220\3753643489.py:59: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  fullDF = pd.concat([fullDF, stationCount])


C:\Users\hec_m\AppData\Local\Temp\ipykernel_15220\3753643489.py:59: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  fullDF = pd.concat([fullDF, stationCount])
C:\Users\hec_m\AppData\Local\Temp\ipykernel_15220\3753643489.py:59: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  fullDF = pd.concat([fullDF, stationCount])
C:\Users\hec_m\AppData\Local\Temp\ipykernel_15220\3753643489.py:59: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version

C:\Users\hec_m\AppData\Local\Temp\ipykernel_15220\3753643489.py:59: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  fullDF = pd.concat([fullDF, stationCount])


C:\Users\hec_m\AppData\Local\Temp\ipykernel_15220\3753643489.py:59: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  fullDF = pd.concat([fullDF, stationCount])
C:\Users\hec_m\AppData\Local\Temp\ipykernel_15220\3753643489.py:59: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  fullDF = pd.concat([fullDF, stationCount])


C:\Users\hec_m\AppData\Local\Temp\ipykernel_15220\3753643489.py:59: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  fullDF = pd.concat([fullDF, stationCount])


C:\Users\hec_m\AppData\Local\Temp\ipykernel_15220\3753643489.py:59: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  fullDF = pd.concat([fullDF, stationCount])


C:\Users\hec_m\AppData\Local\Temp\ipykernel_15220\3753643489.py:59: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  fullDF = pd.concat([fullDF, stationCount])


C:\Users\hec_m\AppData\Local\Temp\ipykernel_15220\3753643489.py:59: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  fullDF = pd.concat([fullDF, stationCount])


,average,count,total,longitude,latitude
id,,,,,
000101@02,1253.431429,350,438701,150251910,62351350
000102@02,1636.831956,363,594170,148779690,61732750
000103@02,20189.041543,337,6803707,149753116,61180770
000104@02,9029.212698,315,2844202,149829440,61076360
000105@02,3000.143731,327,981047,151837810,59804090
...,...,...,...,...,...
090878@55,22253.266667,15,333799,91355397,44919849
130604@55,39833.285714,21,836499,89309085,43055510
180466@55,28767.709677,31,891799,91470611,44794309


In [12]:
len(coords.keys())


9850

In [13]:
import matplotlib.pyplot as plt